In [26]:
import random
import numpy as np
import pandas as pd

data = pd.read_csv("ga_input_0.0.csv")
features = data[["Latitude", "Longitude","Frequency"]].values
features

array([[19.115491, 72.854202, 62.      ],
       [19.058336, 72.830267, 82.      ],
       [19.055292, 72.903055, 48.      ],
       [19.019282, 72.842875, 35.      ],
       [18.933266, 72.836019, 16.      ],
       [19.086521, 72.908956, 53.      ],
       [19.119284, 72.908609, 76.      ],
       [19.096421, 72.862288, 93.      ],
       [19.137622, 72.83192 , 42.      ],
       [19.20838 , 72.842058, 27.      ],
       [18.995706, 72.830745, 65.      ],
       [19.186719, 72.836496, 17.      ],
       [18.925648, 72.82383 , 10.      ],
       [19.123011, 72.908091, 50.      ],
       [18.935955, 72.821655, 27.      ],
       [19.128251, 72.928739, 29.      ],
       [19.102918, 72.887755, 72.      ],
       [19.20892 , 72.978725, 38.      ],
       [19.01394 , 72.826691, 47.      ],
       [19.075768, 73.000236, 89.      ],
       [19.011696, 72.81807 , 20.      ],
       [19.108253, 72.86501 , 56.      ],
       [19.128151, 72.820592, 31.      ],
       [19.107021, 72.827528, 44. 

In [27]:


class ClusterGA:
    def __init__(self, n_clusters, population_size, max_iterations):
        self.n_clusters = n_clusters
        self.population_size = population_size
        self.max_iterations = max_iterations
        
    def initialize_population(self, data):
        self.data = data
        self.population = []
        for i in range(self.population_size):
            clusters = [random.randint(0, self.n_clusters-1) for j in range(len(data))]
            self.population.append(clusters)
            
    def evaluate_fitness(self):
        fitness = []
        for individual in self.population:
            centroids = []
            cluster_frequencies = []
            for i in range(self.n_clusters):
                cluster_data = [self.data[j][:2] for j in range(len(self.data)) if individual[j] == i]
                cluster_freq = [self.data[j][2] for j in range(len(self.data)) if individual[j] == i]
                if len(cluster_data) > 0:
                    centroid = np.mean(cluster_data, axis=0)
                    variance = np.var(cluster_freq)
                else:
                    centroid = np.zeros(2)
                    variance = 0
                centroids.append(centroid)
                cluster_frequencies.append(variance)

            total_distance = 0
            for i in range(len(self.data)):
                cluster_index = individual[i]
                total_distance += np.linalg.norm(self.data[i][:2] - centroids[cluster_index])

            fitness.append(np.sum(cluster_frequencies) / total_distance)

        return fitness

        
    def select_parents(self, fitness):
        max_fitness = max(fitness)
        fitness = [max_fitness - f for f in fitness]
        total_fitness = sum(fitness)
        selection_prob = [f/total_fitness for f in fitness]
        
        parents = []
        for i in range(2):
            r = random.random()
            cum_prob = 0
            for j in range(len(self.population)):
                cum_prob += selection_prob[j]
                if cum_prob >= r:
                    parents.append(self.population[j])
                    break
                    
        return parents
        
    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, len(self.data)-2)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
        
    def mutate(self, individual, mutation_rate):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = random.randint(0, self.n_clusters-1)
        return individual
        
    def run(self, data):
        self.initialize_population(data)
        best_fitness = float('inf')
        
        for i in range(self.max_iterations):
            fitness = self.evaluate_fitness()
            parents = self.select_parents(fitness)
            child1, child2 = self.crossover(parents[0], parents[1])
            child1 = self.mutate(child1, 0.01)
            child2 = self.mutate(child2, 0.01)
            self.population = [parents[0], parents[1], child1, child2] + [self.mutate(parent, 0.01) for parent in self.population[2:]]
            fitness = self.evaluate_fitness()
            best_index = np.argmin(fitness)
            best_clusters = self.population[best_index]
            if fitness[best_index] < best_fitness:
                best_fitness = fitness[best_index]
                best_clusters = self.population[best_index]
            
            print("Generation {}: Best fitness = {}, Best clusters = {}".format(i+1, best_fitness, best_clusters))
            
        
        return best_clusters

# data = np.array([
#     [19.1197 ,72.8464,10], #Andheri
#  [19.0544 ,72.8402,100],#bandra
#  [19.1645, 72.8493,3],#goregaon
#  [19.0669, 72.8774,7],#bkc
#  [19.076 , 72.8846,3],#kurla
#  [19.127 , 72.8291,11],#dnnagar
# ])

data = np.array(features)
ga = ClusterGA(n_clusters=5, population_size=5, max_iterations=100)

best_clusters = ga.run(data)

print("Best clusters found:", best_clusters)

Generation 1: Best fitness = 168.64389752374055, Best clusters = [0, 2, 3, 0, 4, 4, 4, 4, 1, 0, 3, 1, 3, 3, 4, 3, 4, 4, 4, 4, 4, 0, 1, 3, 1, 1, 3, 0, 4, 3, 1, 2, 0, 3, 1, 4, 4, 1, 1, 4, 3, 4, 3, 1, 4, 2, 2, 2, 4, 3, 3, 4, 3, 1, 3, 1, 2, 4, 2, 0, 1, 0, 1, 4, 0, 1, 4, 1, 3, 0, 4, 2, 2, 2, 0, 1, 3, 2, 3, 4, 3, 0, 3, 1, 4, 1, 3, 2, 0, 3, 2, 3, 0, 3, 2, 1, 0, 1, 4, 0, 1, 4, 0, 4, 4, 0, 4, 1, 4, 1, 4, 3, 3, 3, 1, 2, 1, 0, 3, 3, 0, 4, 0, 0, 3, 0, 2, 1, 2, 3, 1, 2, 1, 3, 4, 4, 1, 0, 0, 0, 1, 4, 4, 3, 3, 0, 3, 4, 4]
Generation 2: Best fitness = 168.319882382305, Best clusters = [0, 2, 3, 0, 4, 4, 4, 4, 1, 0, 3, 1, 3, 3, 4, 3, 4, 4, 4, 4, 4, 0, 1, 3, 1, 1, 3, 0, 4, 3, 1, 2, 0, 3, 1, 4, 4, 1, 1, 4, 3, 4, 3, 1, 4, 2, 2, 2, 4, 3, 3, 4, 3, 1, 3, 2, 2, 4, 2, 0, 1, 0, 1, 4, 0, 1, 4, 1, 3, 0, 4, 3, 2, 2, 0, 1, 3, 2, 3, 4, 3, 0, 3, 1, 4, 1, 3, 2, 0, 3, 2, 3, 0, 3, 2, 1, 0, 1, 4, 0, 1, 4, 0, 4, 4, 0, 4, 1, 4, 1, 4, 3, 3, 3, 1, 2, 1, 0, 3, 3, 0, 4, 0, 0, 3, 0, 2, 1, 2, 3, 1, 2, 1, 3, 4, 4, 1, 0, 0, 0, 1, 

In [28]:
print(best_clusters)

[3, 4, 3, 0, 1, 1, 2, 4, 4, 1, 3, 2, 2, 3, 0, 3, 1, 3, 3, 4, 0, 4, 4, 4, 4, 0, 3, 1, 2, 0, 1, 1, 3, 1, 1, 2, 1, 1, 3, 0, 0, 2, 2, 2, 2, 1, 1, 0, 0, 1, 4, 1, 0, 0, 3, 0, 0, 1, 4, 3, 2, 0, 0, 1, 3, 2, 3, 4, 1, 4, 1, 0, 4, 0, 3, 4, 4, 4, 3, 3, 3, 4, 3, 0, 3, 0, 2, 1, 3, 2, 2, 4, 1, 2, 3, 1, 1, 3, 4, 4, 4, 3, 4, 4, 4, 2, 4, 4, 1, 4, 3, 1, 0, 0, 1, 3, 2, 3, 3, 0, 3, 1, 4, 1, 1, 3, 3, 3, 3, 4, 4, 4, 1, 0, 3, 3, 1, 4, 0, 3, 0, 1, 0, 3, 1, 4, 4, 3, 0]


In [31]:
df = pd.read_csv('ga_output_0.0.csv')

In [32]:
df['GeneticCluster'] = best_clusters

In [33]:
df.head()

,Location,Latitude,Longitude,Cluster,Frequency,GeneticCluster
0,Andheri East,19.115491,72.854202,3,62,3
1,Bandra West,19.058336,72.830267,3,82,4
2,Chembur East,19.055292,72.903055,0,48,3
3,Dadar West,19.019282,72.842875,1,35,0
4,Fort,18.933266,72.836019,1,16,1


In [34]:
df = df.rename(columns={'Cluster': 'Kmeans'})
df = df.rename(columns={'GeneticCluster': 'Genetic'})

In [35]:
df.head()

,Location,Latitude,Longitude,Kmeans,Frequency,Genetic
0,Andheri East,19.115491,72.854202,3,62,3
1,Bandra West,19.058336,72.830267,3,82,4
2,Chembur East,19.055292,72.903055,0,48,3
3,Dadar West,19.019282,72.842875,1,35,0
4,Fort,18.933266,72.836019,1,16,1


In [36]:
df.to_csv('ga_output_0.0.csv', index=False)